#### Import

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2 as cv
import os
from keras.models import load_model
from facenet_pytorch import MTCNN

os.environ["CUDA_VISIBLE_DEVICES"]= "1"

tf.debugging.set_log_device_placement(True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

#### Data Adopted & Data Read
- Kaggle Deepfake Detection Challenge Dataset의 **<U>00.zip,01.zip,02.zip</U>**
https://www.kaggle.com/c/deepfake-detection-challenge/data  
- 각각 약 1500개 영상의 REAL, FAKE 정보가 담긴, json 파일로 구성
- 각각 metadata를 Dataframe으로 읽어온 뒤, 하나의 Dataframe으로 구성
- metadata를 sort하여 알파벳 순으로 정렬

In [ ]:
metadata0 = pd.read_json('/content/drive/My Drive/metadata/metadata0.json').T
metadata0 = pd.DataFrame(metadata0["label"])
metadata0.sort_index(inplace=True)

metadata1= pd.read_json('/content/drive/My Drive/metadata/metadata1.json').T
metadata1 = pd.DataFrame(metadata1["label"])
metadata1.sort_index(inplace=True)

metadata2 = pd.read_json('/content/drive/My Drive/metadata/metadata2.json').T
metadata2 = pd.DataFrame(metadata2["label"])
metadata2.sort_index(inplace=True)

In [ ]:
metadata = pd.concat([metadata0, metadata1, metadata2])

In [ ]:
metadata.sort_index(inplace=True)
metadata = metadata.reset_index()
metadata.columns=["filename", "label"]
metadata["label"]=np.where(metadata["label"]=="FAKE", 1, 0)

#### Load Test Video

In [ ]:
test = '/content/drive/My Drive/test_data/'
test_movie_files = [test + x for x in sorted(os.listdir(test))]

#### Load Model 

In [ ]:
model = load_model("/content/drive/My Drive/IR-89-0.0000.h5")
model.summary()

#### Predict Test Video
- v_cap : 영상을 프레임별로 자름
- v_len : 프레임의 총 개수를 변수에 저장
- cv2.VideoCapture.grab() : 프레임을 가져온다. 실패 시 success 변수엔 False가 저장
- cv2.VideoCapture.retrieve() : grab한 프레임을 decode한다. 프레임이 없을 경우 False가 success 변수에 저장
- cv.COLOR_BGR2RGB : 프레임을 BGR에서 RGB로 변환
- MTCNN : 얼굴을 검출하고, margin을 줘 얼굴이 살짝 잘리는 것을 방지
- count : 영상에서 프레임별로 검출되는 모든 얼굴에 대해 번호를 매기기 위함

In [ ]:
%%time
detector = MTCNN(margin=50, keep_all=False, post_process=False, device='cuda:0',thresholds=[.9,.9,.9])
vid_num = 0
scores=[]
filenames = []

for vid in test_movie_files:
    predict_all=[]
    count=0
    file_name_mp4 = vid.split('/')[-1]
    file_name = file_name_mp4.split('.')[0]
    v_cap = cv.VideoCapture(vid)    
    v_len = int(v_cap.get(cv.CAP_PROP_FRAME_COUNT))
    for frm in range(v_len):    
        success = v_cap.grab()
        if frm % 1 == 0:
            success, frame = v_cap.retrieve()
            if not success:
                continue
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            frame = detector(frame)
            if frame is not None:
                frame = np.transpose(frame, (1, 2, 0))
                frame = np.array(cv.resize(np.array(frame),(160 ,160)))
                frame = (frame.flatten() / 255.0).reshape(-1, 160, 160, 3)
                count = count+frame.shape[0]
                predict = model.predict(frame)
                predict_all.append(predict)
            else:
                continue
        else:
            continue
    if (count>11):
        predict_all.sort()
        final_pred = sum(predict_all[5:-5])/(count-10)
        if final_pred == 1: final_pred = [[0.99]]
        elif final_pred == 0: final_pred = [[0.01]]
        scores.append(final_pred[0][0])
        print(" score :",final_pred[0][0]*100)
    else:
        scores.append(0.5)
        print(" score :",50)   
    filenames.append(file_name_mp4)
    print('filename :',file_name_mp4)
v_cap.release()

#### Make Predict Dataframe
- acc를 확인하기 위해 0.5 이상인 값을 1로, 미만인 값을 0으로 변환
- metadata의 label과 df의 predict를 filename를 기준으로 merge

In [ ]:
df = pd.DataFrame({'filename':filenames, 'predict':scores}) 
df["predict"]=np.where(df["predict"]>=0.5, 1, 0)

In [ ]:
predict_df = pd.merge(metadata, df, on='filename')

In [ ]:
compare = (predict_df.label == predict_df.predict)
compare = np.sum(compare)
acc = compare / len(predict.predict)
acc